In [15]:
# !pip -qq install torchtext==0.3.1
# !git clone https://github.com/MiuLab/SlotGated-SLU.git
# !wget -qq https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/week08_multitask/conlleval.py

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [19]:
import numpy as np

import torch, random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    DEVICE = torch.device('cuda:0')
else:
    from torch import FloatTensor, LongTensor
    DEVICE = torch.device('cpu')

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

DEVICE

device(type='cuda', index=0)

In [20]:
import os 

def read_dataset(path):
    with open(os.path.join(path, 'seq.in')) as f_words, \
            open(os.path.join(path, 'seq.out')) as f_tags, \
            open(os.path.join(path, 'label')) as f_intents:
        
        return [
            (words.strip().split(), tags.strip().split(), intent.strip()) 
            for words, tags, intent in zip(f_words, f_tags, f_intents)
        ]
train_data = read_dataset('SlotGated-SLU/data/atis/train/')
val_data = read_dataset('SlotGated-SLU/data/atis/valid/')
test_data = read_dataset('SlotGated-SLU/data/atis/test/')

intent_to_example = {example[2]: example for example in train_data}
# for example in intent_to_example.values():
#     print('Intent:\t', example[2])
#     print('Text:\t', '\t'.join(example[0]))
#     print('Tags:\t', '\t'.join(example[1]))
#     print()
    
from torchtext.data import Field, LabelField, Example, Dataset, BucketIterator

tokens_field = Field()
tags_field = Field(unk_token=None)
intent_field = LabelField()

fields = [('tokens', tokens_field), ('tags', tags_field), ('intent', intent_field)]

train_dataset = Dataset([Example.fromlist(example, fields) for example in train_data], fields)
val_dataset = Dataset([Example.fromlist(example, fields) for example in val_data], fields)
test_dataset = Dataset([Example.fromlist(example, fields) for example in test_data], fields)

tokens_field.build_vocab(train_dataset)
tags_field.build_vocab(train_dataset)
intent_field.build_vocab(train_dataset)

print('Vocab size =', len(tokens_field.vocab))
print('Tags count =', len(tags_field.vocab))
print('Intents count =', len(intent_field.vocab))

train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, val_dataset, test_dataset), batch_sizes=(32, 128, 128), 
    shuffle=True, device=DEVICE, sort=False
)

Vocab size = 869
Tags count = 121
Intents count = 21


In [21]:
class TokenTaggerModel(nn.Module):
    def __init__(self, vocab_size, tags_count, emb_dim=64, lstm_hidden_dim=128, num_layers=2, dropout_p=0.25, pad_idx = 0):
        super().__init__()
        
        self.embeddings_layer = nn.Embedding(vocab_size, emb_dim, padding_idx = pad_idx)
        self.dropout = nn.Dropout(dropout_p)
        self.lstm_layer = nn.LSTM(input_size=emb_dim, hidden_size=lstm_hidden_dim, bidirectional=True, num_layers=num_layers, batch_first=True, dropout = dropout_p if num_layers > 1 else 0)
        self.out_layer = nn.Linear(lstm_hidden_dim * 2, tags_count)

    def forward(self, inputs):
        projections =  self.dropout(self.embeddings_layer(inputs))
        output_lstm, (_, _) = self.lstm_layer(projections)
        
        output = self.dropout(output_lstm)

        output = self.out_layer(output)
        # output = self.out_layer(output_lstm)
        return output

In [22]:
class ModelTrainer_Trigger():
    def __init__(self, model, criterion, optimizer, pad_idx):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.pad_idx = pad_idx
        self.best_valid_loss = float('inf')
        
    def on_epoch_begin(self, is_train, name, batches_count):
        """
        Initializes metrics
        """
        self.epoch_loss = 0
        self.correct_count, self.total_count = 0, 0
        self.is_train = is_train
        self.name = name
        self.batches_count = batches_count
        
        self.model.train(is_train)
        
    def on_epoch_end(self):
        """
        Outputs final metrics
        """
        valid_loss = self.epoch_loss / self.batches_count
        if not(self.is_train) and valid_loss < self.best_valid_loss:
            self.best_valid_loss = valid_loss
            torch.save(self.model, 'TokenTaggerModel.pt')
            
        return '{:>5s} Loss = {:.5f}, Accuracy = {:.2%}'.format(
            self.name, self.epoch_loss / self.batches_count, self.correct_count / self.total_count
        )
        
    def on_batch(self, batch):
        """
        Performs forward and (if is_train) backward pass with optimization, updates metrics
        """
        
        logits = self.model(batch.tokens.transpose(0, 1))
        
        logits = logits.transpose(0, 1)       
        logits = logits.reshape((-1,logits.shape[2]))
        tags = batch.tags.reshape((-1))
        loss = self.criterion(logits, tags)
      
        max_preds = logits.argmax(dim = 1) 
        non_pad_elements = (tags != self.pad_idx).nonzero()
        correct = torch.sum(max_preds[non_pad_elements] == tags[non_pad_elements]).item()
        self.correct_count += correct
        self.total_count += tags[non_pad_elements].shape[0]
        
        if self.is_train:
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
        self.epoch_loss += loss.item()
        
import math
from tqdm import tqdm
tqdm.get_lock().locks = []


def do_epoch(trainer, data_iter, is_train, name=None):
    trainer.on_epoch_begin(is_train, name, batches_count=len(data_iter))
    
    with torch.autograd.set_grad_enabled(is_train):
        with tqdm(total=trainer.batches_count) as progress_bar:
            for i, batch in enumerate(data_iter):
                batch_progress = trainer.on_batch(batch)

                progress_bar.update()
                progress_bar.set_description(batch_progress)
                
            epoch_progress = trainer.on_epoch_end()
            progress_bar.set_description(epoch_progress)
            progress_bar.refresh()

            
def fit(trainer, train_iter, epochs_count=1, val_iter=None):
    best_val_loss = None
    for epoch in range(epochs_count):
        name_prefix = '[{} / {}] '.format(epoch + 1, epochs_count)
        do_epoch(trainer, train_iter, is_train=True, name=name_prefix + 'Train:')
        
        if not val_iter is None:
            do_epoch(trainer, val_iter, is_train=False, name=name_prefix + '  Val:')

In [23]:
tokens_pad_idx = tokens_field.vocab.stoi[tokens_field.pad_token]
tags_pad_idx = tags_field.vocab.stoi[tags_field.pad_token]

model = TokenTaggerModel(vocab_size=len(tokens_field.vocab), tags_count=len(tags_field.vocab), pad_idx = tokens_pad_idx).to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index = tags_pad_idx).to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer_Trigger(model, criterion, optimizer, tags_pad_idx)

fit(trainer, train_iter, epochs_count=30, val_iter=val_iter)

[1 / 30] Train: Loss = 1.57173, Accuracy = 70.71%: 100%|██████████| 140/140 [00:02<00:00, 49.69it/s]
[1 / 30]   Val: Loss = 0.82024, Accuracy = 81.67%: 100%|██████████| 4/4 [00:00<00:00, 37.04it/s]
[2 / 30] Train: Loss = 0.59988, Accuracy = 87.15%: 100%|██████████| 140/140 [00:02<00:00, 50.78it/s]
[2 / 30]   Val: Loss = 0.41893, Accuracy = 90.65%: 100%|██████████| 4/4 [00:00<00:00, 37.74it/s]
[3 / 30] Train: Loss = 0.34554, Accuracy = 92.17%: 100%|██████████| 140/140 [00:02<00:00, 50.78it/s]
[3 / 30]   Val: Loss = 0.27018, Accuracy = 93.76%: 100%|██████████| 4/4 [00:00<00:00, 38.47it/s]
[4 / 30] Train: Loss = 0.23770, Accuracy = 94.45%: 100%|██████████| 140/140 [00:02<00:00, 50.13it/s]
[4 / 30]   Val: Loss = 0.19522, Accuracy = 95.25%: 100%|██████████| 4/4 [00:00<00:00, 36.70it/s]
[5 / 30] Train: Loss = 0.17491, Accuracy = 95.94%: 100%|██████████| 140/140 [00:02<00:00, 50.78it/s]
[5 / 30]   Val: Loss = 0.15624, Accuracy = 96.07%: 100%|██████████| 4/4 [00:00<00:00, 36.36it/s]
[6 / 30] T

In [26]:
model = torch.load('TokenTaggerModel.pt')

do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 0.13111, Accuracy = 97.93%: 100%|██████████| 7/7 [00:00<00:00, 28.34it/s]


In [27]:
from conlleval import evaluate

def eval_tagger(model, test_iter):
    pad_idx = tags_field.vocab.stoi[tags_field.pad_token]
    true_seqs, pred_seqs = [], []

    model.eval()
    with torch.no_grad():
        for batch in test_iter:
            pred = model.forward(batch.tokens.transpose(0, 1))
            pred = pred.transpose(0, 1)
            pred = pred.reshape((-1,pred.shape[2]))
            true = batch.tags.reshape((-1))
            
            max_preds = pred.argmax(dim = 1)
            non_pad_elements = (true != pad_idx).nonzero()
            true = list(map(lambda y: tags_field.vocab.itos[y], true[non_pad_elements].reshape((-1))))
            pred = list(map(lambda y: tags_field.vocab.itos[y], max_preds[non_pad_elements].reshape((-1))))
            pred = ' '.join(pred)
            true = ' '.join(true)
  
            pred_seqs.append(pred)
            true_seqs.append(true)
            
    print('Precision = {:.2f}%, Recall = {:.2f}%, F1 = {:.2f}%'.format(*evaluate(true_seqs, pred_seqs, verbose=False)))

eval_tagger(model, test_iter)   


Precision = 95.28%, Recall = 94.51%, F1 = 94.89%


In [8]:
model = torch.load('TokenTaggerModel.pt')

pad_idx = tags_field.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(model.parameters())

trainer = ModelTrainer_Trigger(model, criterion, optimizer, pad_idx)

do_epoch(trainer, test_iter, is_train=False, name='Test:')

Test: Loss = 6.38924, Accuracy = 97.75%: 100%|██████████| 7/7 [00:00<00:00, 35.90it/s]
